In [1]:
from typing import Dict, List, Optional, Tuple
import dedupe
import pandas as pd
import json
import logging

In [2]:
matched_df_1=pd.read_csv("data/output/matched_df_1.csv")
matched_df_2=pd.read_csv("data/output/matched_df_2.csv")

In [3]:
matched_data_1=matched_df_1.to_dict(orient="index")
matched_data_2=matched_df_2.to_dict(orient="index")

In [4]:
type(matched_data_1)

dict

In [5]:
fields_for_matching=[dedupe.variables.String("business_name"),
                         dedupe.variables.Exact("zip"),
                         dedupe.variables.String("address_clean"),
                         dedupe.variables.String("city_clean"),
                         dedupe.variables.String("state_clean")]

In [6]:
matches=[]
for k,v in matched_data_1.items():
    matches.append((v,matched_data_2[k]))

In [ ]:
training_pairs={
    "match":matches,
    "distinct":[]
}

In [9]:
with open("training_file.json","w") as fp:
    fp.write(json.dumps(training_pairs))